In [1]:
%matplotlib inline
import os
import torch
import random
import pickle
import numpy as np 
import pandas as pd 
from PIL import Image
import torch.nn as nn 
from pathlib import Path
import torch.nn.functional as F
from matplotlib import pyplot as plt 
from tqdm import tqdm, tqdm_notebook
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
d = {'ImageId': df['ImageId'].unique()}
train_df = pd.DataFrame(d)
train_df['ClassesList'] = ['' for i in range(len(train_df))]
train_df['EncodedPixels'] = ['' for i in range(len(train_df))]
for i in range(len(train_df)):
    train_df['ClassesList'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['ClassId'].values.tolist()
    train_df['EncodedPixels'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['EncodedPixels'].values.tolist()

In [3]:
def make_mask( encoded, shape=(1600,256)):
    # Делим на два списка в соответствии с кодировкой
    if isinstance(encoded, str):
        encoded = list(map(int, encoded.split(' ')))
    full,pixel,number = [],[],[]
    [pixel.append(encoded[i]) if i%2==0 else number.append(encoded[i]) for i in range(0, len(encoded))]
    # "Раскрываем" кодировку, получаем индексы закрашенных пикселей
    k=0
    for i in range(len(number)):
        for j in range(number[i]):
            ind = pixel[i]+j
            full.append(ind-1)
        k +=number[i]
    # Создаем массив под готовое изображение    
    mask = np.zeros((1600*256,1), dtype=int)
    # Закрашиваем соответствующие пиксели
    mask[full] = 255
    #преобразем к размерам фотографий металла
    res = np.reshape(mask,(1600, 256)).T
    res = Image.fromarray(res.astype(np.uint8))
    return res

In [4]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [5]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE_1 = 800
RESCALE_SIZE_2 = 128
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [6]:
class SteelDataset(Dataset):
    def __init__(self, names, df, mode):
        super().__init__()
        self.names = names
        self.mode = mode
        if self.mode != 'test':
            self.df = df

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.names)
        
    def __len__(self):
        return self.len_
      
    def load_sample(self, file, mode):
        if mode == 'test':
            image = Image.open('../input/severstal-steel-defect-detection/test_images/'+ file).convert("RGB")
        else:
            image = Image.open('../input/severstal-steel-defect-detection/train_images/'+ file).convert("RGB")
        image.load()
        return image
    
    def __getitem__(self, index):
            transforms_tens = T.Compose([
            T.ToTensor()])

            # загружаем и меняем размер изображения
            img = self._prepare_sample(self.load_sample(self.names[index], self.mode))

            if self.mode == 'test':
                img = np.array(img)
                max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
                img = (img / max_value).astype(np.float32)
                img = transforms_tens(img)
                return img
            
            # загружаем классы и маски
            labels = list(self.df['ClassesList'].loc[self.df['ImageId'] == self.names[index]])[0]
            mask = list(self.df['EncodedPixels'].loc[self.df['ImageId'] == self.names[index]])[0]
            num_obj = len(labels)
            masks = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
            masks = np.transpose(masks,(2, 0, 1))
            for i in range(4):
                for j in range(num_obj):
                    if i == (labels[j]-1):
                # раскодирование масок
                        masks[i] = np.array(self._prepare_sample(make_mask(mask[j])))
                        masks[i] = masks[i] / 255

            # преобазование в тензоры
            masks = torch.as_tensor(masks, dtype=torch.float32)
            label = torch.as_tensor(labels)
            
            # преобразуем изображения и маски
            if self.mode == "train":
                img, masks= transforms_all(img,masks)
                
            img = np.array(img)
            max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
            img = (img / max_value).astype(np.float32)
            img = transforms_tens(img)
            
            return img, masks
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE_1, RESCALE_SIZE_2))
        return image

In [7]:
def transforms_all(image, segmentation):
    if random.random() > 0.5:
        image = TF.autocontrast(image)
    if random.random() > 0.5:
        image = TF.hflip(image)
        segmentation = TF.hflip(segmentation)
    if random.random() > 0.5:
        image = TF.vflip(image)
        segmentation = TF.vflip(segmentation)
    return image, segmentation

In [8]:
from sklearn.model_selection import train_test_split
train_val_names = train_df['ImageId']
train_files,val_files = train_test_split(train_val_names, train_size=0.7)
val_dataset = SteelDataset(list(val_files), train_df,  mode='val')
train_dataset = SteelDataset(list(train_files),train_df, mode='train')

In [11]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)
    
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)        
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

    
class UNet(nn.Module):
    def __init__(self, out_classes=4, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(3, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)
        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)
        # Final Convolution
        self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        return x

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, batch_size):
    model.train()
    running_loss = 0.0
    dice = 0.0
    iou = 0.0
    for X_batch, Y_batch in train_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().numpy() 
        dice += dice_metric(outputs, labels)
        iou += IoU(outputs, labels)
    train_loss = running_loss / len(train_loader)
    train_dice = dice / len(train_loader)
    train_IoU = iou / len(train_loader)
    return train_loss, train_dice, train_IoU

In [ ]:
def eval_epoch(model, val_loader, criterion, batch_size):
    model.eval()
    dice = 0.0
    iou = 0.0
    running_loss = 0.0
    for X_batch, Y_batch in val_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs,labels.long())
            running_loss += loss.item()
            dice += dice_metric(outputs, labels)
            iou += IoU(outputs, labels)
    val_loss = running_loss / len(val_loader)
    val_dice = dice / len(val_loader)
    val_IoU = iou / len(val_loader)
    return val_loss, val_dice, val_IoU

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers = 2, shuffle=True)#,collate_fn = collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,num_workers = 2, shuffle=False)#,collate_fn = collate_fn)
    history = []
    maxIoU = 0
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} train_dice: {t_dice:0.4f} train_IoU: {t_iou:0.4f}\
    \nValidation  val_loss: {v_loss:0.4f} val_dice: {v_dice:0.4f} val_IoU: {v_iou: 0.4f}"
    criterion = nn.BCEWithLogitsLoss()#nn.CrossEntropyLoss()
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        params = model.parameters()
        opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(opt,
                                                   step_size=20,
                                                   gamma=0.1)
        for epoch in range(epochs):
            train_loss, train_dice, train_IoU = fit_epoch(model, train_loader, criterion, opt, batch_size)
            lr_scheduler.step()    
            val_loss, val_dice, val_IoU = eval_epoch(model, val_loader, criterion, batch_size)
            history.append((train_loss, train_dice, train_IoU, val_loss, val_dice, val_IoU))
            if val_IoU > maxIoU:
                maxIoU = val_IoU
                torch.save(model, './weight_unet.dat')
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss, t_dice = train_dice, t_iou = train_IoU, \
                                           v_loss=val_loss, v_dice = val_acc, v_iou = val_IoU ))   
    return history

In [ ]:
def compute_ious(pred, label, classes=[1], ignore_index=255, only_present=True):
    '''computes iou for one ground truth mask and predicted mask'''
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]

def IoU(outputs, labels, classes=None):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
    preds = np.copy(outputs) # copy is imp
    labels = np.array(labels) # tensor to np
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes=[1])))
    iou = np.nanmean(ious)
    return iou

def dice_metric(probability, truth, threshold=0.5, reduction='none'):
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = truth.float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)

        pos_index = torch.nonzero(t_sum >= 1)
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_pos = dice_pos[pos_index]
        dice = dice_pos
    return dice

In [ ]:
model = UNet().to(DEVICE)

In [ ]:
history = train(train_dataset, val_dataset, model, epochs = 40, batch_size = 5)

In [ ]:
train_loss, train_dice, train_IoU, val_loss, val_dice, val_IoU = zip(*history)

In [ ]:
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(30,13))
    ax1.plot(train_loss, label='train', marker='o')
    ax1.plot(val_loss, label='val', marker='o')
    ax1.set_title('Loss per epoch')
    ax1.set_ylabel('loss');
    ax1.set_xlabel('epoch')
    ax1.legend(), ax1.grid()
    
    ax2.plot(train_IoU, label='train_IoU', marker='*')
    ax2.plot(val_IoU, label='val_IoU',  marker='*')
    ax2.set_title('Score per epoch')
    ax2.set_ylabel('mean IoU')
    ax2.set_xlabel('epoch')
    ax2.legend(), ax2.grid()

    ax3.plot(train_dice, label='train_dice', marker='*')
    ax3.plot(val_dice, label='val_dice',  marker='*')
    ax3.set_title('Dice per epoch')
    ax3.set_ylabel('dice')
    ax3.set_xlabel('epoch')
    ax3.legend(), ax3.grid()
    plt.show()

In [70]:
def predict_image_mask_score(model, image, mask, metric = dice_metric):
    model.eval()
    
    image = image.to(DEVICE)
    mask = mask.to(DEVICE)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
        score = metric(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score

In [73]:
def score(model, dataset, metric = dice_metric):
    res_score = []
    for i in tqdm(range(len(val_dataset))):
        img, mask = dataset[i]
        pred_mask, score = predict_image_mask_score(model, img, mask, metric)
        res_score.append(score)
    return res_score

In [17]:
model = torch.load('../input/unet-weight/weight_unet_wew.dat')

In [74]:
print(np.mean(score(model, val_dataset)))

In [71]:
k1,k2 = int(np.random.uniform(0,100)),int(np.random.uniform(0,100))
image1, mask1 = val_dataset[k1]
pred_mask1, score1 = predict_image_mask_score(model, image1, mask1)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,13))
ax1.imshow(np.rollaxis(image1.numpy(), 0, 3))
ax1.set_title('Picture');
ax1.set_axis_off()

ax2.imshow(mask1)
ax2.set_title('Ground truth')
ax2.set_axis_off()

ax3.imshow(pred_mask1)
ax3.set_title('UNet | '+ str(score1))
ax3.set_axis_off()

image2, mask2 = val_dataset[k2]
pred_mask2, score2 = predict_image_mask_score(model, image2, mask2)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,13))
ax1.imshow(np.rollaxis(image2.numpy(), 0, 3))
ax1.set_title('Picture');
ax1.set_axis_off()

ax2.imshow(mask2)
ax2.set_title('Ground truth')
ax2.set_axis_off()

ax3.imshow(pred_mask2)
ax3.set_title('UNet | '+str(score2))
ax3.set_axis_off()